In [24]:
from bs4 import BeautifulSoup as bs
import pandas as pd
import undetected_chromedriver as webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.select import Select
from selenium.common import exceptions 
from time import sleep
from tqdm import tqdm
import re

In [25]:
t = pd.read_excel(r"C:\Users\nikit\Downloads\Тестовое задание.xlsx", sheet_name = ['Москва', 'Ростов-на-Дону']) # Считываение имен товаров из Excel 
Mos = t['Москва'].iloc[2:,0].to_list() # Сохраняем в список
Moscl = []
def goods(): # Функция,  которой названия товаров очищаются от имен фирм типо Самокат, Лавка, Озон и тд, если фирма производтель, то ее не трогаем.
    for i in Mos:
        str = i.replace('+', ' ')
        str = re.sub(r'\s+',' ', str.replace('Самокат ', ''))
        if str[-2:] == ' г':
            str = (re.search(r'[\S ]+(?=[,]\s\d+\s[г])', str))
            str = str.group(0)
        Moscl.append(str)
    return Moscl
def get_df(): # Функция создающая датафрейм, в котором будет информация о товарах 
    df =  pd.DataFrame(columns = ['Лавка','Озон','Вкус','ЦЛ', 'ЦЛП','ЦО', 'ЦОП','ЦВ', 'ЦВП'], index = Mos)
    return df

In [26]:
def transliterate(name):
    if name.split(' ')[-1] == 'г':
        name = name[:-2]
   # Слоаврь с заменами
    slovar = {'а':'a','б':'b','в':'v','г':'g','д':'d','е':'e','ё':'yo',
      'ж':'zh','з':'z','и':'i','й':'j','к':'k','л':'l','м':'m','н':'n',
      'о':'o','п':'p','р':'r','с':'s','т':'t','у':'u','ф':'f','х':'h',
      'ц':'c','ч':'ch','ш':'sh','щ':'sch','ъ':'','ы':'y','ь':'','э':'e',
      'ю':'u','я':'ya', 'А':'A','Б':'B','В':'V','Г':'G','Д':'D','Е':'E','Ё':'YO',
      'Ж':'ZH','З':'Z','И':'I','Й':'I','К':'K','Л':'L','М':'M','Н':'N',
      'О':'O','П':'P','Р':'R','С':'S','Т':'T','У':'U','Ф':'F','Х':'H',
      'Ц':'C','Ч':'CH','Ш':'SH','Щ':'SCH','Ъ':'','Ы':'y','Ь':'','Э':'E',
      'Ю':'U','Я':'YA',',':'','?':'','~':'','!':'','@':'','#':'',
      '$':'','%':'','^':'','*':'','(':'',')':'','-':'','=':'',
      ':':'',';':'','<':'','>':'','\'':'','"':'','\\':'','/':'','№':'',
      '[':'',']':'','{':'','}':'','ґ':'','ї':'', 'є':'','Ґ':'g','Ї':'i',
      'Є':'e', '—':'', ' ':'_', '+':' '}
        
   # Циклически заменяем все буквы в строке
    for key in slovar:
        name = name.lower().replace(key, slovar[key])
    return name

In [27]:
browser = webdriver.Chrome() #запускается объект браузера, в котором потом будут происходить все действия
df = get_df() #создается датафрейм, в котором будут храниться все наименования и цены товаров
Moscl = goods()

In [62]:
class Ozon:
    searchstr = '/html/body/div[1]/div/div[1]/header/div[1]/div/div[2]/div/div/form/div/div[2]/input'
    price = 'div[1]/div[1]/div/span[1]'
    full_price = 'div[1]/div[1]/div/span[2]'
    name = 'div[1]/a/div/span'
    check = '/html/body/div[1]/div/div[1]/header/div[1]/div/div[2]/div/div/form/div/div[2]/input'
    selector_of_supply_el = '#paginatorContent > div > div > div > div'
    url_ozon = 'https://www.ozon.ru/category/supermarket-25000/?__rr=1&miniapp=supermarket'
    button_address = '/html/body/div[1]/div/div[1]/div[1]/div/div/div/div/div/div/div[1]/button'
    address_string = '/html/body/div[4]/div/div[2]/div/div/div/div/div/div/div[1]/div/div/div/div[1]/div/div/form/div/div/fieldset/div[1]/div/div/div/label/div[1]/div/textarea'
    cross_address_string = '/html/body/div[4]/div/div[2]/div/div/div/div/div/div/div[1]/div/div/div/div[1]/div/div/form/div/div/fieldset/div[1]/div/div/div/label/div[1]/div/div/div/button/span'
    first_submit_button = '/html/body/div[5]/div/div/div/div'
    second_submit_button = '/html/body/div[4]/div/div[2]/div/div/div/div/div/div/div[1]/div/div/div/div[2]/button'
    def __init__(self, list, location, browser, df):
        self.names = list
        self.location = location
        self.browser = browser
        self.df = df
    def open_site(self):
        """Открывает сайт магазина"""
        self.browser.get(self.url_ozon)
    def set_location(self):
        """В случае если в брауезере на сайте не выставлен адрес произойдет его установка"""
        # to-do Проверка локации на сайте, если она совпадает с введенной хотя бы отчасти, то не трогаем и скипаем,
        # иначе все что снизу + при первом заходе на сайт, выставление геолокации для обычного озона
        try:
            x_but = wait(self.browser, 10).until(EC.presence_of_element_located((By.XPATH, self.button_address)))
            sleep(0.1)
            x_but.click()
            adrstro = wait(self.browser, 10).until(EC.presence_of_element_located((By.XPATH, self.address_string)))
            wait(self.browser, 15).until(EC.presence_of_element_located((By.XPATH, self.cross_address_string))).click()
            adrstro.send_keys(self.location)
            sleep(0.5)
            adrstro.send_keys(Keys.ENTER)
            sleep(1)
            wait(self.browser, 15).until(EC.presence_of_element_located((By.XPATH, self.first_submit_button))).click()
            sleep(2)
            save_but = wait(self.browser, 10).until(EC.element_to_be_clickable((By.XPATH, self.second_submit_button))).click()
            sleep(0.1)
            save_but.click()
        except exceptions.TimeoutException:
            print('Проблема установки адреса, впишите его в ручную, проверьте, что адрес введен на Ozon fresh и введите в консоль какой либо символ')
            input()
        except Exception as ex:
            print(f'Ошибка по неизвестной причине, код ошибки {str(ex)}')
    def sup(self):
        """Находит карточки всех доступных товаров, по данному запросу"""
        for k in range(len(self.names)):
            i = Moscl[k]
            searchbar = wait(self.browser, 10).until(EC.presence_of_element_located((By.XPATH, self.searchstr)))
            searchbar.click()
            if k != 0:
                searchbar.click()
                searchbar.clear()
            searchbar.send_keys(i)
            searchbar.send_keys(Keys.ENTER)
            wait(self.browser, 15).until(EC.text_to_be_present_in_element_value((By.XPATH, self.searchstr), i))
            try:
                t = self.browser.find_elements(By.CSS_SELECTOR, self.selector_of_supply_el)
            except:
                try:
                    t = self.browser.find_elements(By.CSS_SELECTOR, self.selector_of_supply_el)
                except: 
                    print("Ошибка поиска товаров")
                    break
            c = 0
            nmt = transliterate(i)
            for lt in t:
                link = lt.find_element(By.TAG_NAME, 'a').get_attribute('href')
                price = lt.find_element(By.XPATH, self.price).text
                full_price = lt.find_element(By.XPATH, self.full_price).text
                name = lt.find_element(By.XPATH, self.name).text
                if c == 0:
                    price_of_best = price 
                    full_price_of_best = full_price
                    name_of_best = name 
                    c += 1
                try:
                    nameinlink = link.split(r'/')[-1]
                    if nmt.split().issubset(nameinlink.lower()) or nameinlink.split().lower().issubset(nmt):
                        price_of_best = price 
                        full_price_of_best = full_price
                        name_of_best = name 
                        break
                except:
                    break
            self.df.iloc[k, 1] = name_of_best
            self.df.iloc[k, 5] = price_of_best
            self.df.iloc[k, 6] = full_price_of_best

In [65]:
print(df)

                                                                                                Лавка  \
Бекон сырокопчёный Дымов нарезка, 150 г                      Дымов Бекон сырокопченый, нарезка, 200 г   
Из Эквадора: Бананы Самокат 4 шт.                                                  Бананы, 0,4 - 1 кг   
Брауни FitnesShock 15% протеина, без сахара, го...  FitnesShock Протеиновое печенье Брауни без сах...   
Бумага туалетная Самокат 2 слоя, 8 рулонов          Туалетная бумага Zewa Плюс Сирень, 2 слоя, 8 р...   
Бумага туалетная Zewa Natural Comfort белая, 3 ...  Туалетная бумага Zewa Natural comfort Без аром...   
Вода минеральная Borjomi газированная, в стекле...  Вода газированная Славянская Жемчужина минерал...   
Вода питьевая Святой Источник негазированная, 5...  Вода негазированная Святой Источник промо-пак,...   
Гуляш Самокат говядина, охлаждённый, 380 г                      Гуляш из говядины, охлажденный, 400 г   
Драже M&M’s с арахисом и молочным шоколадом, 145 г  Кон

In [64]:
location = 'Москва, Запорожская улица, 5'
ozon = Ozon(Moscl, location, browser, df)
ozon.open_site()
# ozon.set_location()
ozon.sup()

##### Старый вариант, неработает из за того, что объединил в класс, если разобрать должно быть нормально

In [ ]:
class Ozon:
    searchstr = '/html/body/div[1]/div/div[1]/header/div[1]/div/div[2]/div/div/form/div/div[2]/input'
    price = '/div[1]/div[1]/div/span[1]'
    full_price = 'div[1]/div[1]/div/span[2]/text()'
    name = 'div[1]/a/div/span'
    check = '/html/body/div[1]/div/div[1]/header/div[1]/div/div[2]/div/div/form/div/div[2]/input'
    class_of_supply_el = 'j8j_23'
    url_ozon = 'https://www.ozon.ru/category/supermarket-25000/?__rr=1&miniapp=supermarket'
    button_address = '/html/body/div[1]/div/div[1]/div[1]/div/div/div/div/div/div/div[1]/button'
    address_string = '/html/body/div[4]/div/div[2]/div/div/div/div/div/div/div[1]/div/div/div/div[1]/div/div/form/div/div/fieldset/div[1]/div/div/div/label/div[1]/div/textarea'
    cross_address_string = '/html/body/div[4]/div/div[2]/div/div/div/div/div/div/div[1]/div/div/div/div[1]/div/div/form/div/div/fieldset/div[1]/div/div/div/label/div[1]/div/div/div/button/span'
    first_submit_button = '/html/body/div[5]/div/div/div/div'
    second_submit_button = '/html/body/div[4]/div/div[2]/div/div/div/div/div/div/div[1]/div/div/div/div[2]/button'
    def __init__(self, list, location):
        self.names = list
        self.location = location
    def open_site(url_ozon):
        browser.get(url_ozon)
    def location():
        try:
            x_but = wait(browser, 10).until(EC.presence_of_element_located((By.XPATH, button_address)))
            sleep(0.1)
            x_but.click()
            adrstro = wait(browser, 10).until(EC.presence_of_element_located((By.XPATH, address_string)))
            wait(browser, 15).until(EC.presence_of_element_located((By.XPATH, cross_address_string))).click()
            adrstro.send_keys(self.location)
            sleep(0.5)
            adrstro.send_keys(Keys.ENTER)
            sleep(1)
            wait(browser, 15).until(EC.presence_of_element_located((By.XPATH, first_submit_button))).click()
            sleep(2)
            #wait(browser, 10).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[4]/div/div[2]/div/div/div/div/div/div/div[1]/div/div/div/div[2]/button'))).click()
            save_but = wait(browser, 10).until(EC.presence_of_element_located((By.XPATH, second_submit_button)))
            sleep(0.1)
            save_but.click()
        except:
            pass
    def sup(searchstr, check, class_of_supply_el):
        for k in range(len(Moscl)):
            i = Moscl[k]
            searchbar = wait(browser, 10).until(EC.presence_of_element_located((By.XPATH, searchstr)))
            searchbar.click()
            searchbar.send_keys(Keys.CONTROL + 'a' + Keys.DELETE)
            searchbar.send_keys(i)
            searchbar.send_keys(Keys.ENTER)
            wait(browser, 15).until(EC.text_to_be_present_in_element_value((By.XPATH, check), i))
            print(wait(browser, 15).until(EC.text_to_be_present_in_element_value((By.XPATH, check), i)))
            t = browser.find_elements(By.CLASS_NAME, class_of_supply_el)
            bestl(t, i, k, price, full_price, name, weight)
    def bestl(t, nm, k, price, full_price, name, weight):
        nmt = transliterate(nm)
        c = 0
        best = ''
        for lt in t:
            link = lt.find_element(By.TAG_NAME, 'a').get_attribute('href')
            price = lt.find_element(By.XPATH, price).text
            full_price = lt.find_element(By.XPATH, full_price).text
            name = lt.find_element(By.XPATH, name).text
            if c == 0:
                print(name)
                price_of_best = price 
                full_price_of_best = full_price
                name_of_best = name 
                weight_best = weight
                c += 1
            try:
                nameinlink = link.split(r'/')[-1]
                if nmt.split().issubset(nameinlink.lower()) or nameinlink.split().lower().issubset(nmt):
                    price_of_best = price 
                    full_price_of_best = full_price
                    name_of_best = name 
                    weight_best = weight
                    break
            except:
                break
        df.iloc[k, 0] = name
        df.iloc[k, 3] = price_of_best
        df.iloc[k, 4] = full_price_of_best
    

In [21]:
from time import sleep
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.select import Select
from selenium.common import exceptions 
import undetected_chromedriver as webdriver
searchstr = '/html/body/div[1]/div/div[1]/header/div[1]/div/div[2]/div/div/form/div/div[2]/input'
price = '/div[1]/div[1]/div/span[1]'
full_price = 'div[1]/div[1]/div/span[2]/text()'
name = 'div[1]/a/div/span'
check = '/html/body/div[1]/div/div[1]/header/div[1]/div/div[2]/div/div/form/div/div[2]/input'
class_of_supply_el = 'j8j_23'
url_ozon = 'https://www.ozon.ru/category/supermarket-25000/?__rr=1&miniapp=supermarket'
button_address = '/html/body/div[1]/div/div[1]/div[1]/div/div/div/div/div/div/div[1]/button'
address_string = '/html/body/div[4]/div/div[2]/div/div/div/div/div/div/div[1]/div/div/div/div[1]/div/div/form/div/div/fieldset/div[1]/div/div/div/label/div[1]/div/textarea'
cross_address_string = '/html/body/div[4]/div/div[2]/div/div/div/div/div/div/div[1]/div/div/div/div[1]/div/div/form/div/div/fieldset/div[1]/div/div/div/label/div[1]/div/div/div/button/span'
first_submit_button = '/html/body/div[5]/div/div/div/div'
second_submit_button = '/html/body/div[4]/div/div[2]/div/div/div/div/div/div/div[1]/div/div/div/div[2]/button'


browser = webdriver.Chrome()

browser.get(url_ozon)

x_but = wait(browser, 10).until(EC.presence_of_element_located((By.XPATH, button_address)))
sleep(0.1)
x_but.click()
adrstro = wait(browser, 10).until(EC.presence_of_element_located((By.XPATH, address_string)))
wait(browser, 15).until(EC.presence_of_element_located((By.XPATH, cross_address_string))).click()
adrstro.send_keys(location)
sleep(0.5)
adrstro.send_keys(Keys.ENTER)
sleep(1)
wait(browser, 15).until(EC.presence_of_element_located((By.XPATH, first_submit_button))).click()
sleep(2)
#wait(browser, 10).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[4]/div/div[2]/div/div/div/div/div/div/div[1]/div/div/div/div[2]/button'))).click()
save_but = wait(browser, 10).until(EC.presence_of_element_located((By.XPATH, second_submit_button)))
sleep(0.1)
save_but.click()

TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x010BC1C3+27395]
	(No symbol) [0x01053DC4]
	(No symbol) [0x00F51B7F]
	(No symbol) [0x00F92C65]
	(No symbol) [0x00F92D3B]
	(No symbol) [0x00FCEC82]
	(No symbol) [0x00FB39E4]
	(No symbol) [0x00FCCB24]
	(No symbol) [0x00FB3736]
	(No symbol) [0x00F87541]
	(No symbol) [0x00F880BD]
	GetHandleVerifier [0x01373A93+2876371]
	GetHandleVerifier [0x013C7F5D+3221661]
	GetHandleVerifier [0x0113D634+556916]
	GetHandleVerifier [0x0114474C+585868]
	(No symbol) [0x0105CE04]
	(No symbol) [0x01059818]
	(No symbol) [0x010599B7]
	(No symbol) [0x0104BF0E]
	BaseThreadInitThunk [0x75837BA9+25]
	RtlInitializeExceptionChain [0x7702C10B+107]
	RtlClearBits [0x7702C08F+191]


In [ ]:
# browser.get(url_ozon)
# sleep(1)
# try:
#     x_but = wait(browser, 10).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/div/div[1]/div[1]/div/div/div/div/div/div/div[1]/button')))
#     sleep(0.1)
#     x_but.click()
#     adrstro = wait(browser, 10).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[4]/div/div[2]/div/div/div/div/div/div/div[1]/div/div/div/div[1]/div/div/form/div/div/fieldset/div[1]/div/div/div/label/div[1]/div/textarea')))
#     wait(browser, 15).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[4]/div/div[2]/div/div/div/div/div/div/div[1]/div/div/div/div[1]/div/div/form/div/div/fieldset/div[1]/div/div/div/label/div[1]/div/div/div/button/span'))).click()
#     adrstro.send_keys('Москва, Запорожская улица, 5')
#     sleep(0.5)
#     adrstro.send_keys(Keys.ENTER)
#     sleep(1)
#     wait(browser, 15).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[5]/div/div/div/div'))).click()
#     sleep(2)
#     wait(browser, 10).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[4]/div/div[2]/div/div/div/div/div/div/div[1]/div/div/div/div[2]/button'))).click()
#     save_but = wait(browser, 10).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[4]/div/div[2]/div/div/div/div/div/div/div[1]/div/div/div/div[2]/button')))
#     sleep(0.1)
#     save_but.click()
# except:
#     pass

In [ ]:
# searchstr = '/html/body/div[1]/div/div[1]/header/div[1]/div/div[2]/div/div/form/div/div[2]/input'
# price = '#paginatorContent > div > div > div:nth-child(1) > div > div.jj7_23 > div.i7x_23.xi7_23.c306-a.c306-b3 > div > span.c306-a1.tsHeadline500Medium.c306-c0'
# full_price = '#paginatorContent > div > div > div:nth-child(1) > div > div.jj7_23 > div.i7x_23.xi7_23.c306-a.c306-b3 > div > span.c306-a1.tsBodyControl400Small.c306-b0'
# name = '#paginatorContent > div > div > div > div > div.jj7_23 > a > div > span'
# check = '/html/body/div[1]/div/div[1]/header/div[1]/div/div[2]/div/div/form/div/div[2]/input'
# class_sup = 'j8j_23'
# sup(searchstr, check, class_sup)

In [ ]:
# def sup(searchstr, check, class_of_supply_el, weight = '', m =''):
#     for k in range(len(Moscl)):
#         i = Moscl[k]
#         searchbar = wait(browser, 10).until(EC.presence_of_element_located((By.XPATH, searchstr)))
#         searchbar.click()
#         searchbar.send_keys(Keys.CONTROL + 'a' + Keys.DELETE)
#         searchbar.send_keys(i)
#         print(i)
#         if m != 'l':
#             searchbar.send_keys(Keys.ENTER)
#         wait(browser, 15).until(EC.text_to_be_present_in_element_value((By.XPATH, check), i))
#         print(wait(browser, 15).until(EC.text_to_be_present_in_element_value((By.XPATH, check), i)))
#         t = browser.find_elements(By.CLASS_NAME, class_of_supply_el)
#         bestl(t, i, k, price, full_price, name, weight)

In [ ]:
# def bestl(t, nm, k, price, full_price, name, weight):
#     nmt = transliterate(nm)
#     c = 0
#     best = ''
#     print(t)
#     for lt in t:
#         print(1)
#         link = lt.find_element(By.TAG_NAME, 'a').get_attribute('href')
#         price = lt.find_element(By.CSS_SELECTOR, price).text
#         full_price = lt.find_element(By.CSS_SELECTOR, full_price).text
#         name = lt.find_element(By.CSS_SELECTOR, name).text
#         if weight != '':
#             weight = lt.find_element(By.XPATH, weight).text
#         if c == 0:
#             print(name)
#             price_of_best = price 
#             full_price_of_best = full_price
#             name_of_best = name 
#             weight_best = weight
#             c += 1
#         try:
#             nameinlink = link.split(r'/')[-1]
#             if nmt.split().issubset(nameinlink.lower()) or nameinlink.split().lower().issubset(nmt):
#                 price_of_best = price 
#                 full_price_of_best = full_price
#                 name_of_best = name 
#                 weight_best = weight
#                 break
#         except:
#             break
#     if weight != '':
#         df.iloc[k, 0] = name + ', ' + weight
#     else:
#         df.iloc[k, 0] = name
#     df.iloc[k, 3] = price_of_best
#     df.iloc[k, 4] = full_price_of_best